In [1]:
ENV["COLUMNS"] = 120

using Distributions
using HypothesisTests

# minlik版のFisher検定の実装では ≤ ではなく, 次の ⪅ を使う必要がある.
x ⪅ y = x < y || x ≈ y

# minlik版Fisher検定のP値の素朴な(高率の悪い)実装
function fisher_test(a, b, c, d)
    hg = Hypergeometric(a+b, c+d, a+c)
    pa = pdf(hg, a)
    sum(pdf(hg, i) for i in support(hg) if pdf(hg, i) ⪅ pa)
end

# ≤ を使う正しくない実装
function fisher_test_le(a, b, c, d)
    hg = Hypergeometric(a+b, c+d, a+c)
    pa = pdf(hg, a)
    sum(pdf(hg, i) for i in support(hg) if pdf(hg, i) ≤ pa)
end

fisher_test_le (generic function with 1 method)

In [2]:
# https://qiita.com/WolfMoon/items/530413ce7c8439d18c18

using SpecialFunctions

# 以下も不適切な実装
function fisher(a, b, c, d)
    lchoose(n, k) = logfactorial(n) - logfactorial(k) - logfactorial(n - k)
    Stats(i, e, f, g, n) = exp(lchoose(e, i) + lchoose(f, g - i) - lchoose(n, g))
    e, f, g, h, n = a + b, c + d, a + c, b + d, a + b + c + d
    mi = max(0, e + g - n)
    length = min(e, g) - mi
    prob = [Stats(mi + i, e, f, g, n) for i in 0:length]
    #println("Fisher's Exact Test for Count Data (two tailed)")
    #println("p value =", sum(prob[prob .<= Stats(a, e, f, g, n)]))
    sum(prob[prob .<= Stats(a, e, f, g, n)])
end;

In [3]:
s, t, n = 11, 9, 10
hg = Hypergeometric(s, t, n)
supp = support(hg)
Any[
    [(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher(a, s-a, n-a, t-n+a) for a in supp];;
    [pvalue(FisherExactTest(a, s-a, n-a, t-n+a)) for a in supp];;
    [fisher_test(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test_le(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher(a, s-a, n-a, t-n+a) ≈ pvalue(FisherExactTest(a, s-a, n-a, t-n+a)) for a in supp];;
    [fisher(a, s-a, n-a, t-n+a) ≈ fisher_test(a, s-a, n-a, t-n+a) for a in supp];;
    [fisher_test(a, s-a, n-a, t-n+a) ≈ fisher_test_le(a, s-a, n-a, t-n+a) for a in supp];;
]

10×8 Matrix{Any}:
 (1, 10, 9, 0)  0.000119076  0.000119076  0.000119076  0.000119076   true   true   true
 (2, 9, 8, 1)   0.00547749   0.00547749   0.00547749   0.00279829    true   true  false
 (3, 8, 7, 2)   0.0697785    0.0697785    0.0697785    0.0697785     true   true   true
 (4, 7, 6, 3)   0.36985      0.36985      0.36985      0.36985       true   true   true
 (5, 6, 5, 4)   1.0          1.0          1.0          1.0           true   true   true
 (6, 5, 4, 5)   0.684925     1.0          1.0          0.684925     false  false  false
 (7, 4, 3, 6)   0.36985      0.36985      0.36985      0.36985       true   true   true
 (8, 3, 2, 7)   0.0697785    0.0697785    0.0697785    0.0697785     true   true   true
 (9, 2, 1, 8)   0.00279829   0.00547749   0.00547749   0.00547749   false  false   true
 (10, 1, 0, 9)  0.000119076  0.000119076  0.000119076  0.000119076   true   true   true

In [4]:
(a, b, c, d) = (6, 5, 4, 5)
@show fisher(a, b, c, d)
@show pvalue(FisherExactTest(a, b, c, d))
@show fisher_test(a, b, c, d)
@show fisher_test_le(a, b, c, d);

fisher(a, b, c, d) = 0.6849249821386099
pvalue(FisherExactTest(a, b, c, d)) = 0.9999999999999998
fisher_test(a, b, c, d) = 1.0
fisher_test_le(a, b, c, d) = 0.6849249821386045


In [5]:
s, t, n = 11, 9, 10
hg = Hypergeometric(s, t, n)
supp = support(hg)
[(a, pdf(Hypergeometric(s, t, n), a)) for a in supp]

10-element Vector{Tuple{Int64, Float64}}:
 (1, 5.953798523457966e-5)
 (2, 0.0026792093355560843)
 (3, 0.032150512026673)
 (4, 0.15003572279114077)
 (5, 0.3150750178613957)
 (6, 0.31507501786139563)
 (7, 0.15003572279114077)
 (8, 0.032150512026673)
 (9, 0.0026792093355560865)
 (10, 5.953798523457966e-5)